In [ ]:
import autotrader.datamodule as dm
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [ ]:
# Using DataClient class
client = dm.DataClient(api_key=__import__('os').environ['EOD_API_KEY'])

In [ ]:
# Using get_data and analyser functions
nifty_data = client.get_data(symbol='NIFTY')
bnf_data = client.get_data(symbol='BANKNIFTY')
nifty_daily_data = dm.analyser(nifty_data, frequency='D')
bnf_daily_data = dm.analyser(bnf_data, frequency='D')
nifty_weekly_data = dm.analyser(nifty_data, frequency='W-THU')
bnf_weekly_data = dm.analyser(bnf_data, frequency='W-THU')
nifty_monthly_data = dm.analyser(nifty_data, frequency='M-THU')
bnf_monthly_data = dm.analyser(bnf_data, frequency='M-THU')

In [ ]:
# Using ratio_analysis function
rolling_days = 45
ratio_data = dm.ratio_analysis(bnf_weekly_data, nifty_weekly_data, add_rolling=rolling_days)
px.line(ratio_data, x=ratio_data.index, y=['BANKNIFTY Change', 'NIFTY Change', f'Rolling {rolling_days} Ratio'], color_discrete_map={'BANKNIFTY Change': 'red', 'NIFTY Change': 'blue', f'Rolling {rolling_days} Ratio': 'green'})

In [ ]:
# Using gambler function
for index in ['FINNIFTY']:
    print(f'{index}\n')
    data = client.get_data(symbol=index)
    for frequency in ['D', 'D-TUE', 'W-TUE', 'M-TUE']:
        print(f'{frequency}\n')
        dm.gambler(data, frequency, 'abs_change')

In [ ]:
dm.analyser(data, frequency='W-TUE')

In [ ]:
# Modelling IV surface
vol_surface = pd.read_csv('data/vol_surface.csv', index_col=0)
vol_surface['tte_in_days'] = (vol_surface.time_to_expiry * 365).astype(int)

In [ ]:
# Modelling IV surface
vol_surface_dict = {}
for tte in vol_surface.tte_in_days.unique():
    X = vol_surface.loc[vol_surface.tte_in_days == tte][['distance', 'distance_squared']]
    y = vol_surface.loc[vol_surface.tte_in_days == tte]['iv_multiple']
    model = LinearRegression()
    model.fit(X, y)
    dis_sq_coeff, dis_coeff, intercept = model.coef_[1], model.coef_[0], model.intercept_
    score = model.score(X, y)
    if score > 0.9:
        vol_surface_dict[tte/365] = {'dis_sq_coeff': dis_sq_coeff, 'dis_coeff': dis_coeff, 'intercept': intercept, 'score': score}
    print(f'{tte} days to expiry: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')
vol_surface_weights = pd.DataFrame(vol_surface_dict).T.reset_index().rename(columns={'index': 'time_to_expiry'})
vol_surface_weights.sort_values('time_to_expiry', inplace=True)

In [ ]:
# Modelling IV surface - Distance Squared coefficient vs Time to Expiry (inverse)
for param in np.arange(0.2, 1.5, 0.1):
    vol_surface_weights['tte_inverse'] = 1 / (vol_surface_weights.time_to_expiry**param)
    X = vol_surface_weights['tte_inverse'].values.reshape(-1, 1)
    y = vol_surface_weights['dis_coeff']
    model = LinearRegression()
    model.fit(X, y)
    print(f'{param} param: Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')

In [ ]:
# Modelling IV surface - Distance coefficient vs Time to Expiry
X = vol_surface_weights['time_to_expiry'].values.reshape(-1, 1)
y = vol_surface_weights['dis_coeff']
model = LinearRegression()
model.fit(X, y)
print(f'Coefficients: {model.coef_}, Intercept: {model.intercept_}, R2: {model.score(X, y)}')

In [ ]:
px.scatter(vol_surface_weights, x='time_to_expiry', y='dis_coeff')

In [ ]:
def coefficients_for_surface(tte):

    # distance squared coefficient
    dfs2 = ((1 / tte ** 0.5) * 55.156423) - 168.2251168
    dfs2 = min(dfs2, 20000)

    # distance coefficient
    dfs = 1 / ((tte ** 0.45) * 5)
    dfs = min(dfs, 5)
    dfs = -6 + dfs
    return dfs2, dfs, .98

In [ ]:
coefficients_for_surface(1.25/365)